In [73]:
#code for part 1 of data extraction

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, StaleElementReferenceException

# List of URLs to visit
urls = [
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
]

# Initialize the WebDriver
driver = webdriver.Chrome()

# Initialize a DataFrame to store the extracted data
df = pd.DataFrame(columns=['route_link', 'bus_route', 'source_url'])

def extract_data():
    global df
    try:
        routes = driver.find_elements(By.XPATH, "//a[@class='route']")
        href_l = []
        title_l = []
        for route in routes:
            href = route.get_attribute("href")  # Extract the href attribute (URL)
            title = route.get_attribute("title")  # Extract the title attribute
            href_l.append(href)
            title_l.append(title)

        # Append extracted data to the DataFrame
        temp_df = pd.DataFrame({
            'route_link': href_l, 
            'bus_route': title_l, 
            'source_url': driver.current_url
        })
        df = pd.concat([df, temp_df], ignore_index=True)
        print(f"Data extracted from {driver.current_url}")

    except NoSuchElementException as e:
        print(f"Error in extracting data: {e}")

def process_url(url):
    driver.get(url)
    
    # Extract data from the first page
    extract_data()

    # Find the pagination container
    try:
        pagination_container = driver.find_element(By.CLASS_NAME, 'DC_117_paginationTable')
        page_tabs = pagination_container.find_elements(By.CLASS_NAME, 'DC_117_pageTabs')
        total_pages = len(page_tabs)
        print(f"Total pages found for {url}: {total_pages}")

        # Track processed pages to avoid reprocessing
        processed_pages = set()

        for i in range(total_pages):
            if i in processed_pages:
                print(f"Page {i+1} already processed for {url}. Skipping.")
                continue

            # Click the page tab
            try:
                page_tab = page_tabs[i]
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable(page_tab))
                driver.execute_script("arguments[0].click();", page_tab)
                
                # Wait for the page to load
                time.sleep(3)  # Adjust if needed based on the page load time

                # Extract data from the current page
                extract_data()
                
                # Mark this page as processed
                processed_pages.add(i)

            except ElementClickInterceptedException:
                print(f"Element click intercepted on page {i+1} for {url}. Retrying with JavaScript click.")
                driver.execute_script("arguments[0].click();", page_tab)

            except StaleElementReferenceException:
                print(f"Stale element reference on page {i+1} for {url}. Re-fetching elements.")
                page_tabs = pagination_container.find_elements(By.CLASS_NAME, 'DC_117_pageTabs')
                continue

            except NoSuchElementException:
                print(f"Page tab for page {i+1} not found for {url}.")
                break

    except NoSuchElementException:
        print(f"Pagination container not found for {url}.")
    except Exception as e:
        print(f"An unexpected error occurred for {url}: {e}")

# Process each URL in the list
for url in urls:
    process_url(url)

# Save the DataFrame to a CSV file in the specified directory
try:
    file_path = r'C:\Users\Bhuvanesh\Desktop\Yogesh Guvi\Redbus Project\Scrapping\Final\bus_route_information.csv'  # Update file path as needed
    df.to_csv(file_path, index=False)
    print(f"Data saved to {file_path}")
except PermissionError as e:
    print(f"PermissionError: {e}. Please ensure the file is not open and the path is correct.")
finally:
    # Close the browser
    driver.quit()


Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Total pages found for https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile: 5
Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Total pages found for https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile: 2
Data extracted from https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile
Data extracted from https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometil

In [86]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import os
import logging
from urllib.parse import urlparse

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

route_links = [
    "https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad",
    "https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada",
    "https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad"
]  # Sample URLs, replace or add more as needed

# Define the path to save the output CSV file
# csv_output_path = "C:\Users\Bhuvanesh\Desktop\Yogesh Guvi\Redbus Project\Scrapping\Final\", 'bus_information.csv')
csv_output_path = r'C:\Users\Bhuvanesh\Desktop\Yogesh Guvi\Redbus Project\Scrapping\Final\bus_information.csv'
# Initialize the WebDriver
driver = webdriver.Chrome()

def get_bus_route(url):
    """Extract the bus route from the URL."""
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.strip('/').split('/')
    return path_parts[-1] if path_parts else "unknown_route"

def scroll_to_bottom(times=1):
    """Scroll to the bottom of the page multiple times using JavaScript."""
    for _ in range(times):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.loading-spinner"))
        )
        time.sleep(2)  # Shorter wait time

def extract_data(route_link, bus_route):
    """Extract data from the current page and include the route link and bus route."""
    try:
        # Wait for the page to load and elements to be present
        WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.travels.lh-24.f-bold.d-color"))
        )

        # Initialize data lists
        data_selectors = {
            "bus_name": "div.travels.lh-24.f-bold.d-color",
            "bus_type": "div.bus-type.f-12.m-top-16.l-color.evBus",
            "departure_time": "div.dp-time.f-19.d-color.f-bold",
            "duration": "div.dur.l-color.lh-24",
            "arrival_time": "div.bp-time.f-19.d-color.disp-Inline",
            "star_rating": "div.lh-18.rating.rat-green",
            "price": "div.fare.d-block",
            "seats_available": "div.seat-left.m-top-30"
        }

        extracted_data = {key: [elem.text for elem in driver.find_elements(By.CSS_SELECTOR, selector)]
                          for key, selector in data_selectors.items()}

        # Debugging: Print lengths of extracted lists
        for key, values in extracted_data.items():
            logging.info(f"Found {len(values)} {key.replace('_', ' ')}.")

        # Create DataFrame from the collected data
        max_length = max(len(values) for values in extracted_data.values())
        for key in extracted_data:
            extracted_data[key] += [""] * (max_length - len(extracted_data[key]))

        # Add the route_link and bus_route to the DataFrame
        extracted_data["route_link"] = [route_link] * len(extracted_data["bus_name"])
        extracted_data["bus_route"] = [bus_route] * len(extracted_data["bus_name"])

        bus_table = pd.DataFrame(extracted_data)

        return bus_table

    except TimeoutException as e:
        logging.error(f"TimeoutException: {e}")
        return pd.DataFrame()

    except NoSuchElementException as e:
        logging.error(f"NoSuchElementException: {e}")
        return pd.DataFrame()

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return pd.DataFrame()

def navigate_and_extract_once():
    all_bus_data = []

    try:
        # Click the button with class "button"
        try:
            action_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            action_button.click()
            WebDriverWait(driver, 10).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, "div.loading-spinner"))
            )
            time.sleep(2)  # Shorter wait time

            # Scroll to the bottom of the page twice
            scroll_to_bottom(times=2)

            # Extract data from the resulting page
            bus_table_next = extract_data(route_link, bus_route)
            if not bus_table_next.empty:
                all_bus_data.append(bus_table_next)

        except NoSuchElementException as e:
            logging.error(f"No button with class 'button' found or button not clickable: {e}")

    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

    # Combine all data into a single DataFrame
    if all_bus_data:
        combined_bus_data = pd.concat(all_bus_data, ignore_index=True)
    else:
        combined_bus_data = pd.DataFrame()

    return combined_bus_data

def process_urls(urls):
    all_bus_data = []

    for url in urls:
        try:
            driver.get(url)
            global route_link
            global bus_route
            route_link = url
            bus_route = get_bus_route(url)

            # Navigate through pages and extract data
            bus_table = navigate_and_extract_once()
            if not bus_table.empty:
                all_bus_data.append(bus_table)

        except Exception as e:
            logging.error(f"An error occurred while processing URL {url}: {e}")

    # Combine all data into a single DataFrame
    if all_bus_data:
        combined_bus_data = pd.concat(all_bus_data, ignore_index=True)
    else:
        combined_bus_data = pd.DataFrame()

    return combined_bus_data

# Process URLs and extract data
combined_bus_data = process_urls(route_links)

# Save the DataFrame to a CSV file
if not combined_bus_data.empty:
    combined_bus_data.to_csv(csv_output_path, index=False)
    logging.info(f"Data saved to {csv_output_path}")
else:
    logging.info("No data to save.")

# Close the browser
driver.quit()


2024-08-21 23:18:52,681 - INFO - Found 15 bus name.
2024-08-21 23:18:52,681 - INFO - Found 15 bus type.
2024-08-21 23:18:52,681 - INFO - Found 15 departure time.
2024-08-21 23:18:52,681 - INFO - Found 15 duration.
2024-08-21 23:18:52,681 - INFO - Found 15 arrival time.
2024-08-21 23:18:52,681 - INFO - Found 14 star rating.
2024-08-21 23:18:52,681 - INFO - Found 15 price.
2024-08-21 23:18:52,681 - INFO - Found 7 seats available.
2024-08-21 23:18:52,697 - INFO - Data saved to C:\Users\Bhuvanesh\Documents\New folder\bus_information.csv


In [80]:
#Data Base creation

import mysql.connector

mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    auth_plugin="mysql_native_password"
)

mycursor=mydb.cursor()

mycursor.execute("create DATABASE redbus_project")

mycursor.execute("show databases")
for x in mycursor:
    print(x)


mycursor.close()
mydb.close()

DatabaseError: 1007 (HY000): Can't create database 'redbus_project'; database exists

In [43]:
#Data Base creation

import mysql.connector

mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    auth_plugin="mysql_native_password",
    database="redbus_project"
)

mycursor=mydb.cursor()

query_1="""
CREATE TABLE bus_route_information (
    id INT AUTO_INCREMENT PRIMARY KEY, 
    route_link VARCHAR(250), 
    bus_route Varchar(250),
    source_url Varchar(250))"""

try:
    mycursor.execute(query_1)
    mydb.commit()
    print("Table created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    mycursor.close()
    mydb.close()

Table created successfully.


In [ ]:
import mysql.connector

# Connection details
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    database="redbus_project"
)

mycursor = mydb.cursor()

# Correct SQL query to create the table
query_2 = """
CREATE TABLE bus_information (
    id INT AUTO_INCREMENT PRIMARY KEY,
    bus_name VARCHAR(250),
    bus_type VARCHAR(250),
    departure_time TIME,
    duration VARCHAR(100),
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seats_available INT,
    bus_route VARCHAR(100),
    route_link VARCHAR(250)
)
"""

try:
    mycursor.execute(query_2)
    mydb.commit()  # Make sure to use 'mydb.commit()' instead of 'conn.commit()'
    print("Table created successfully.")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    mycursor.close()
    mydb.close()


In [88]:
import pandas as pd

df1=pd.read_csv(r"C:\Users\Bhuvanesh\Desktop\Yogesh Guvi\Redbus Project\Scrapping\Final\bus_route_information.csv")
print(df1)
print(df1.isnull().sum())
print(df1.info())

                                            route_link  \
0    https://www.redbus.in/bus-tickets/vijayawada-t...   
1    https://www.redbus.in/bus-tickets/hyderabad-to...   
2    https://www.redbus.in/bus-tickets/kakinada-to-...   
3    https://www.redbus.in/bus-tickets/visakhapatna...   
4    https://www.redbus.in/bus-tickets/chittoor-and...   
..                                                 ...   
453  https://www.redbus.in/bus-tickets/haldia-to-ko...   
454  https://www.redbus.in/bus-tickets/kolkata-to-p...   
455  https://www.redbus.in/bus-tickets/kolkata-to-a...   
456  https://www.redbus.in/bus-tickets/habra-to-kol...   
457  https://www.redbus.in/bus-tickets/habra-to-con...   

                                  bus_route  \
0                   Vijayawada to Hyderabad   
1                   Hyderabad to Vijayawada   
2                 Kakinada to Visakhapatnam   
3                 Visakhapatnam to Kakinada   
4    Chittoor (Andhra Pradesh) to Bangalore   
..                   

In [11]:
df2=pd.read_csv(r"C:\Users\Bhuvanesh\Desktop\Yogesh Guvi\Redbus Project\Scrapping\Final\bus_information.csv")
print(df2.isnull().sum())
print(df2.info())
df2

bus_name              0
bus_type              0
departure_time        0
duration              0
reaching_time         0
star_rating        7668
price                 0
seats_available    7002
bus_route             0
route_link            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13714 entries, 0 to 13713
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bus_name         13714 non-null  object 
 1   bus_type         13714 non-null  object 
 2   departure_time   13714 non-null  object 
 3   duration         13714 non-null  object 
 4   reaching_time    13714 non-null  object 
 5   star_rating      6046 non-null   float64
 6   price            13714 non-null  object 
 7   seats_available  6712 non-null   object 
 8   bus_route        13714 non-null  object 
 9   route_link       13714 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.0+ MB
None


,bus_name,bus_type,departure_time,duration,reaching_time,star_rating,price,seats_available,bus_route,route_link
0,APSRTC - 2538,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:15,04h 00m,4:15,4.4,INR 281,30 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
1,APSRTC - 2837,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:30,03h 00m,3:30,4.7,INR 281,34 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
2,APSRTC - 23551,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:45,04h 00m,4:45,4.1,INR 281,29 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,APSRTC - 2536,INDRA(A.C. Seater),1:15,03h 15m,4:30,4.3,INR 340,35 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,APSRTC - 23537,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",1:30,04h 00m,5:30,4.3,INR 281,34 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...,...,...,...,...,...,...,...
13709,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,06h 10m,19:55,NaN,INR 595,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13710,City Queen Bus Service,Non AC Seater (2+2),20:50,08h 10m,5:00,NaN,INR 250,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13711,City Queen Bus Service,Non AC Seater (2+2),21:30,10h 00m,7:30,NaN,INR 285,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13712,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,NaN,INR 87,NaN,habra-to-kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...


In [13]:
#Remove str from price column

def clean_price(price_str):
    # Remove currency symbol and commas, and convert to numeric
    return pd.to_numeric(price_str.replace('INR ', '').replace(',', ''), errors='coerce')

# Apply the function to the 'price' column
df2['price'] = df2['price'].apply(clean_price)

df2

,bus_name,bus_type,departure_time,duration,reaching_time,star_rating,price,seats_available,bus_route,route_link
0,APSRTC - 2538,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:15,04h 00m,4:15,4.4,281.0,30 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
1,APSRTC - 2837,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:30,03h 00m,3:30,4.7,281.0,34 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
2,APSRTC - 23551,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:45,04h 00m,4:45,4.1,281.0,29 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,APSRTC - 2536,INDRA(A.C. Seater),1:15,03h 15m,4:30,4.3,340.0,35 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,APSRTC - 23537,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",1:30,04h 00m,5:30,4.3,281.0,34 Seats available,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...,...,...,...,...,...,...,...
13709,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,06h 10m,19:55,NaN,595.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13710,City Queen Bus Service,Non AC Seater (2+2),20:50,08h 10m,5:00,NaN,250.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13711,City Queen Bus Service,Non AC Seater (2+2),21:30,10h 00m,7:30,NaN,285.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13712,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,NaN,87.0,NaN,habra-to-kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...


In [15]:
#Remove str from seats_available column

def clean_seats(seats_str):
    import re
    if isinstance(seats_str, str):  # Ensure the input is a string
        # Extract numbers and convert to integer
        match = re.search(r'\d+', seats_str)  # Search for digits in the string
        if match:
            return int(match.group(0))  # Return the first matched number
    return None  # Return None if no number is found or input is not a string

# Apply the function to the 'seats_available' column
df2['seats_available'] = df2['seats_available'].apply(clean_seats)

df2

,bus_name,bus_type,departure_time,duration,reaching_time,star_rating,price,seats_available,bus_route,route_link
0,APSRTC - 2538,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:15,04h 00m,4:15,4.4,281.0,30.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
1,APSRTC - 2837,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:30,03h 00m,3:30,4.7,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
2,APSRTC - 23551,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:45,04h 00m,4:45,4.1,281.0,29.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,APSRTC - 2536,INDRA(A.C. Seater),1:15,03h 15m,4:30,4.3,340.0,35.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,APSRTC - 23537,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",1:30,04h 00m,5:30,4.3,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...,...,...,...,...,...,...,...
13709,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,06h 10m,19:55,NaN,595.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13710,City Queen Bus Service,Non AC Seater (2+2),20:50,08h 10m,5:00,NaN,250.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13711,City Queen Bus Service,Non AC Seater (2+2),21:30,10h 00m,7:30,NaN,285.0,NaN,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13712,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,NaN,87.0,NaN,habra-to-kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...


In [25]:
#Treating Null values

df2.fillna({
    'rating': 0,  # Example placeholder for NaN in 'rating'
    'seats_available': 0,   
    # Add other columns if necessary
}, inplace=True)
df2

,bus_name,bus_type,departure_time,duration,reaching_time,star_rating,price,seats_available,bus_route,route_link
0,APSRTC - 2538,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:15,04h 00m,4:15,4.4,281.0,30.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
1,APSRTC - 2837,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:30,03h 00m,3:30,4.7,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
2,APSRTC - 23551,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:45,04h 00m,4:45,4.1,281.0,29.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,APSRTC - 2536,INDRA(A.C. Seater),1:15,03h 15m,4:30,4.3,340.0,35.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,APSRTC - 23537,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",1:30,04h 00m,5:30,4.3,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...,...,...,...,...,...,...,...
13709,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,06h 10m,19:55,0.0,595.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13710,City Queen Bus Service,Non AC Seater (2+2),20:50,08h 10m,5:00,0.0,250.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13711,City Queen Bus Service,Non AC Seater (2+2),21:30,10h 00m,7:30,0.0,285.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13712,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,0.0,87.0,0.0,habra-to-kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...


In [27]:
def clean_nan(value):
    """ Convert NaN values to 0 """
    return 0 if pd.isna(value) else value

# Handle NaN values
df2 = df2.applymap(clean_nan)
df2

C:\Users\Bhuvanesh\AppData\Local\Temp\ipykernel_22068\4116061347.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df2 = df2.applymap(clean_nan)


,bus_name,bus_type,departure_time,duration,reaching_time,star_rating,price,seats_available,bus_route,route_link
0,APSRTC - 2538,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:15,04h 00m,4:15,4.4,281.0,30.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
1,APSRTC - 2837,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:30,03h 00m,3:30,4.7,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
2,APSRTC - 23551,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",0:45,04h 00m,4:45,4.1,281.0,29.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,APSRTC - 2536,INDRA(A.C. Seater),1:15,03h 15m,4:30,4.3,340.0,35.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,APSRTC - 23537,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",1:30,04h 00m,5:30,4.3,281.0,34.0,kakinada-to-visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
...,...,...,...,...,...,...,...,...,...,...
13709,WBTC (CTC) KOLKATA - ASANSOL - PURULIA (AC VOL...,Volvo AC Seater (2+2),13:45,06h 10m,19:55,0.0,595.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13710,City Queen Bus Service,Non AC Seater (2+2),20:50,08h 10m,5:00,0.0,250.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13711,City Queen Bus Service,Non AC Seater (2+2),21:30,10h 00m,7:30,0.0,285.0,0.0,kolkata-to-purulia,https://www.redbus.in/bus-tickets/kolkata-to-p...
13712,WBTC (CTC) HABRA-DIGHA via Bally - 26|12:45,Non AC Seater (2+3),12:45,02h 55m,15:40,0.0,87.0,0.0,habra-to-kolaghat,https://www.redbus.in/bus-tickets/habra-to-kol...


In [182]:
#store bus_route_information into SQL


import mysql.connector
import pandas as pd

# Establish database connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    database="redbus_project"
)

# Create a cursor object
mycursor = mydb.cursor()

# Define the insert query
insert_query = """
INSERT INTO bus_route_information (route_link, bus_route, source_url)
VALUES (%s, %s, %s)
"""

# Sample DataFrame for illustration
# df1 should have columns 'route_link', 'bus_route', and 'source_url'

# Insert data into the table
for _, row in df1.iterrows():
    mycursor.execute(insert_query, (row['route_link'], row['bus_route'], row['source_url']))

# Commit the transaction
mydb.commit()

# Close the cursor and connection
mycursor.close()
mydb.close()


In [29]:
#store bus_information into SQL

import mysql.connector
import pandas as pd

# Establish database connection
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    database="redbus_project"
)

# Create a cursor object
mycursor = mydb.cursor()

# Define the insert query
insert_query = """
INSERT INTO bus_information (
    bus_name, bus_type, departure_time, duration, reaching_time,
    star_rating, price, seats_available, bus_route, route_link
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Sample DataFrame for illustration
# Ensure df1 has the correct columns and data types


# Insert data into the table
for _, row in df2.iterrows():
    mycursor.execute(insert_query, (
        row['bus_name'], row['bus_type'], row['departure_time'],
        row['duration'], row['reaching_time'], row['star_rating'],
        row['price'], row['seats_available'], row['bus_route'],
        row['route_link']
    ))

# Commit the transaction
mydb.commit()

# Close the cursor and connection
mycursor.close()
mydb.close()


In [31]:
import mysql.connector

# Database connection details
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    database="redbus_project"
)
cursor = conn.cursor()

# Query to select data from the table
query = "SELECT * FROM bus_route_information LIMIT 10"

cursor.execute(query)

# Fetch and print the data
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()


(1, 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'Vijayawada to Hyderabad', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
(2, 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada', 'Hyderabad to Vijayawada', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
(3, 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam', 'Kakinada to Visakhapatnam', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
(4, 'https://www.redbus.in/bus-tickets/visakhapatnam-to-kakinada', 'Visakhapatnam to Kakinada', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
(5, 'https://www.redbus.in/bus-tickets/chittoor-andhra-pradesh-to-bangalore', 'Chittoor (Andhra Pradesh) to Bangalore', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile')
(6, 'https://www.redbus.in/bus-tickets/kadapa-to-bangalore', 'Kadapa to Bangalore', 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchomet

In [33]:
import mysql.connector

# Database connection details
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="yogeshwaran",
    database="redbus_project"
)
cursor = conn.cursor()

# Query to select data from the table
query = "SELECT * FROM bus_information LIMIT 10"

cursor.execute(query)

# Fetch and print the data
rows = cursor.fetchall()
for row in rows:
    print(row)

cursor.close()
conn.close()


(26, 'APSRTC - 2538', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=900), '04h 00m', datetime.timedelta(seconds=15300), 4.4, Decimal('281.00'), 30, 'kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam')
(27, 'APSRTC - 2837', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=1800), '03h 00m', datetime.timedelta(seconds=12600), 4.7, Decimal('281.00'), 34, 'kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam')
(28, 'APSRTC - 23551', 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)', datetime.timedelta(seconds=2700), '04h 00m', datetime.timedelta(seconds=17100), 4.1, Decimal('281.00'), 29, 'kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-tickets/kakinada-to-visakhapatnam')
(29, 'APSRTC - 2536', 'INDRA(A.C. Seater)', datetime.timedelta(seconds=4500), '03h 15m', datetime.timedelta(seconds=16200), 4.3, Decimal('340.00'), 35, 'kakinada-to-visakhapatnam', 'https://www.redbus.in/bus-